### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Functions

In [2]:
def downcaste_dtype(df):
    previous_memory_consumption=(df.memory_usage().sum())/1024**2 
    for col in df.columns:
        col_dtype=df[col].dtypes
        if(col_dtype!='object'):
            xmin=df[col].min()
            xmax=df[col].max()
            if(str(df[col].dtype)[0]=='i'): 
                if(np.iinfo('int8').min<xmin and np.iinfo('int8').max>xmax):
                    df[col]=df[col].astype('int8')
                    
                elif(np.iinfo('int16').min<xmin and np.iinfo('int16').max>xmax):
                    df[col]=df[col].astype('int16')
                    
                elif(np.iinfo('int32').min<xmin and np.iinfo('int32').max>xmax):
                    df[col]=df[col].astype('int32')
                    
                elif(np.iinfo('int64').min<xmin and np.iinfo('int64').max>xmax):
                    df[col]=df[col].astype('int64')
                    
            elif(str(df[col].dtype)[0]=='f'): 
                if(np.finfo('float16').min<xmin and np.finfo('float16').max>xmax):
                    df[col]=df[col].astype('float16')
                    
                elif(np.finfo('float32').min<xmin and np.finfo('float32').max>xmax):
                    df[col]=df[col].astype('float32')
                    
                elif(np.finfo('float64').min<xmin and np.finfo('float64').max>xmax):
                    df[col]=df[col].astype('float64')
            else:pass
    After_memory_consumption=(df.memory_usage().sum())/1024**2
    percentage_of_decrease=((previous_memory_consumption-After_memory_consumption)/previous_memory_consumption)*100
    print('Memory usage before downcasting: {:.2f}\n'.format(previous_memory_consumption))
    print('After Downcasting the memory usage decreased by: {:.2f}\n'.format(After_memory_consumption))
    print('Memory usage decreased by {:.3f}%'.format(percentage_of_decrease)) 

### Load Data

In [3]:
cred_bal = pd.read_csv("DataSet/credit_bureau_balance.csv")
loan_train = pd.read_csv("DataSet/loan_applications_train.csv")
prev_credit_cards = pd.read_csv("DataSet/previous_credit_cards.csv")
prev_credits = pd.read_csv("DataSet/previous_credits.csv")
prev_loan_apps = pd.read_csv("DataSet/previous_loan_applications.csv")
prev_pos_cash_loans = pd.read_csv("DataSet/previous_pos_cash_loans.csv")
repayment_hist = pd.read_csv("DataSet/repayment_history.csv")

### Reduce Memory usage

In [4]:
print("Optimizing cred_bal table: ")
downcaste_dtype(cred_bal)
print("Optimizing loan_train table: ")
downcaste_dtype(loan_train)
print("Optimizing prev_credit_cards table: ")
downcaste_dtype(prev_credit_cards)
print("Optimizing prev_credits_table: ")
downcaste_dtype(prev_credits)
print("Optimizing prev_loan_apps: ")
downcaste_dtype(prev_loan_apps)
print("Optimizing prev_pos_cash_loans: ")
downcaste_dtype(prev_pos_cash_loans)
print("Optimizing repayment_hist: ")
downcaste_dtype(repayment_hist)

Optimizing cred_bal table: 
Memory usage before downcasting: 624.85

After Downcasting the memory usage decreased by: 338.46

Memory usage decreased by 45.833%
Optimizing loan_train table: 
Memory usage before downcasting: 286.23

After Downcasting the memory usage decreased by: 92.38

Memory usage decreased by 67.725%
Optimizing prev_credit_cards table: 
Memory usage before downcasting: 673.88

After Downcasting the memory usage decreased by: 289.33

Memory usage decreased by 57.065%
Optimizing prev_credits_table: 
Memory usage before downcasting: 222.62

After Downcasting the memory usage decreased by: 112.95

Memory usage decreased by 49.265%
Optimizing prev_loan_apps: 
Memory usage before downcasting: 471.48

After Downcasting the memory usage decreased by: 309.01

Memory usage decreased by 34.459%
Optimizing prev_pos_cash_loans: 
Memory usage before downcasting: 610.43

After Downcasting the memory usage decreased by: 238.45

Memory usage decreased by 60.937%
Optimizing repayment_

In [5]:
prev_credits = pd.merge(cred_bal, prev_credits, on='sk_id_bureau')


merged_data = pd.merge(loan_train, prev_credits, on='sk_id_curr', how='left')


bureau_agg = prev_credits.groupby('sk_id_curr').agg({'amt_credit_sum': 'sum',
                                                     'amt_credit_max_overdue': 'max',
                                                     'amt_credit_sum_debt': 'sum',
                                                     'amt_credit_sum_limit': 'sum'}).reset_index()

                                                    
credit_cards_agg = prev_credit_cards.groupby('sk_id_curr').agg({'amt_balance': 'sum',
                                                                'amt_credit_limit_actual': 'max',
                                                                'amt_drawings_atm_current': 'sum'}).reset_index()


loan_app_agg = prev_loan_apps.groupby('sk_id_curr').agg({'amt_annuity': 'sum',
                                                          'amt_application': 'sum',
                                                          'amt_credit': 'sum'}).reset_index()


pos_cash_loans_agg = prev_pos_cash_loans.groupby('sk_id_curr').agg({'cnt_instalment': 'sum'}).reset_index()


repayment_hist_agg = repayment_hist.groupby('sk_id_curr').agg({'amt_instalment': 'sum',
                                                               'amt_payment': 'sum'}).reset_index()


In [6]:
merged_data = pd.merge(merged_data, bureau_agg, on='sk_id_curr', how='left')
merged_data = pd.merge(merged_data, credit_cards_agg, on='sk_id_curr', how='left')
merged_data = pd.merge(merged_data, loan_app_agg, on='sk_id_curr', how='left')
merged_data = pd.merge(merged_data, pos_cash_loans_agg, on='sk_id_curr', how='left')
merged_data = pd.merge(merged_data, repayment_hist_agg, on='sk_id_curr', how='left')

In [7]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14916892 entries, 0 to 14916891
Columns: 153 entries, sk_id_curr to amt_payment
dtypes: float16(64), float32(21), float64(7), int16(2), int32(2), int8(37), object(20)
memory usage: 6.7+ GB


In [8]:
merged_data

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit_x,amt_annuity_x,...,amt_credit_sum_limit_y,amt_balance,amt_credit_limit_actual,amt_drawings_atm_current,amt_annuity,amt_application,amt_credit_y,cnt_instalment,amt_instalment,amt_payment
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,127954.257812,NaN,NaN,NaN,9251.775391,179055.0,179055.0,456.0,2.196257e+05,2.196257e+05
1,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,127954.257812,NaN,NaN,NaN,9251.775391,179055.0,179055.0,456.0,2.196257e+05,2.196257e+05
2,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,127954.257812,NaN,NaN,NaN,9251.775391,179055.0,179055.0,456.0,2.196257e+05,2.196257e+05
3,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,127954.257812,NaN,NaN,NaN,9251.775391,179055.0,179055.0,456.0,2.196257e+05,2.196257e+05
4,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,127954.257812,NaN,NaN,NaN,9251.775391,179055.0,179055.0,456.0,2.196257e+05,2.196257e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14916887,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.000000,NaN,NaN,NaN,166203.140625,2902167.0,3395448.0,1618.0,3.068389e+06,3.525820e+06
14916888,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.000000,NaN,NaN,NaN,166203.140625,2902167.0,3395448.0,1618.0,3.068389e+06,3.525820e+06
14916889,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.000000,NaN,NaN,NaN,166203.140625,2902167.0,3395448.0,1618.0,3.068389e+06,3.525820e+06
14916890,456255,0,Cash loans,F,N,N,0,157500.0,675000.0,49117.5,...,0.000000,NaN,NaN,NaN,166203.140625,2902167.0,3395448.0,1618.0,3.068389e+06,3.525820e+06


In [9]:
merged_data.to_csv("DataSet/merged_data.csv")